## 구현 순서

1. SageMaker를 이용한 MNIST 모델 훈련
2. SageMaker 훈련모델 배포
3. Neo를 사용한 훈련 모델 배포
4. 추론 컨테이너를 지정한 Neo 훈련 모델 배포


# 1. SageMaker를 이용한 MNIST 모델 훈련

### 환경 설정

In [1]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_region_name

In [2]:
tf_version = '1.15.2'

In [3]:
training_data_uri = "s3://sagemaker-sample-data-{}/tensorflow/mnist".format(region)

In [4]:
# mnist 훈련 스크립트
training_script = "mnist.py"

### SageMaker 훈련 작업 생성

In [5]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    # MNIST 훈련 script
    entry_point=training_script,
    role=role,
    instance_count=1,
    instance_type="ml.p2.xlarge",
    framework_version=tf_version,
    py_version="py3",
)

In [ ]:
# Sagemaker 훈련 작업 시작
estimator.fit(training_data_uri)

# 2. SageMaker 훈련모델 배포

In [7]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.c5.xlarge")

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


---!

### 훈련 데이터 다운로드

In [8]:
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_data.npy train_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_labels.npy train_labels.npy

train_data = np.load("train_data.npy")
train_labels = np.load("train_labels.npy")

download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/train_data.npy to ./train_data.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/train_labels.npy to ./train_labels.npy


## 첫 번째 실험

- Neo Compile 전 훈련 모델 성능 측정

In [9]:
%%time
# %%time : 해당 셀이 실행되는 데 걸리는 시간 측정

# prediction : 모델의 예측값
# label : 데이터의 실제값
# count : 예측이 틀린 횟수

count = 0
for i in range(0, 10000):
    data = train_data[i]
    predictions = predictor.predict(data)
    prediction = predictions["predictions"][0]["classes"]
    label = train_labels[i]
#     print("prediction: {}, label: {}, matched: {}".format(prediction, label, prediction == label))
    if (prediction != label):
        count += 1
count = 100 - count/10000
print("Accuracy: {}".format(count))

Accuracy: 99.966
CPU times: user 22.6 s, sys: 577 ms, total: 23.2 s
Wall time: 2min 19s


### 엔드포인트 삭제

In [10]:
predictor.delete_endpoint()

# 3. Neo를 사용한 훈련 모델 배포

### 모델 컴파일

In [11]:
# output_path 컴파일된 모델이 S3에 저장되는 위치
output_path = "/".join(estimator.output_path.split("/")[:-1])

# Neo compile할 대상 하드웨어 및 프레임워크 선택
optimized_estimator = estimator.compile_model(
    target_instance_family="ml_c5",
    input_shape={"data": [1, 784]},  # Batch size 1, 1 channel, 28*28 image size.
    output_path=output_path,
    framework="tensorflow",
    framework_version="1.15.3",
)

???????????????????????................................!

### 컴파일 된 모델 엔드포인트에 배포

In [12]:
# 모델 최적화 후 실시간으로 예측 요청을 제공하는 Endpoint에 배포
optimized_predictor = optimized_estimator.deploy(
    initial_instance_count=1, instance_type="ml.c5.xlarge"
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!

## 두 번째 실험

- Neo compile 후 / 추론 컨테이너 이미지 지정 전 훈련 모델 성능 측정

In [13]:
%%time
# %%time : 해당 셀이 실행되는 데 걸리는 시간 측정

# prediction : 모델의 예측값
# label : 데이터의 실제값
# count : 예측이 틀린 횟수

count = 0
for i in range(0, 10000):
    data = train_data[i].tolist()
    predictions = optimized_predictor.predict({"instances": np.asarray(data).tolist()})
    prediction = predictions["predictions"][0]["classes"]
    label = train_labels[i]
#     print("prediction: {}, label: {}, matched: {}".format(prediction, label, prediction == label))
    if (prediction != label):
        count += 1
count = 100 - count/10000
print("Accuracy: {}".format(count))

Accuracy: 99.966
CPU times: user 22.3 s, sys: 624 ms, total: 23 s
Wall time: 1min 19s


### 엔드포인트 삭제

In [14]:
optimized_predictor.delete_endpoint()

# 4. 추론 컨테이너를 지정한 Neo 훈련 모델 배포

### 모델 컴파일

In [15]:
# output_path 컴파일된 모델이 S3에 저장되는 위치
output_path = "/".join(estimator.output_path.split("/")[:-1])

# Neo compile할 대상 하드웨어 및 프레임워크 선택
optimized_estimator = estimator.compile_model(
    target_instance_family="ml_c5",
    input_shape={"data": [1, 784]},  # Batch size 1, 1 channel, 28*28 image size.
    output_path=output_path,
    framework="tensorflow",
    framework_version="1.15.3",
)

??????????????????????????.................................!

### 추론 컨테이너 이미지 지정

In [16]:
optimized_estimator.image_uri = (
    "301217895009.dkr.ecr.us-west-2.amazonaws.com/sagemaker-inference-tensorflow:1.15.3-cpu-py3"
)

### 컴파일 된 모델 엔드포인트에 배포

In [17]:
optimized_predictor = optimized_estimator.deploy(
    initial_instance_count=1, instance_type="ml.c5.xlarge"
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!

## 세 번째 실험

- Neo compile 후 / 추론 컨테이너 이미지 지정 후 훈련 모델 성능 측정

In [18]:
%%time
# %%time : 해당 셀이 실행되는 데 걸리는 시간 측정

# prediction : 모델의 예측값
# label : 데이터의 실제값
# count : 예측이 틀린 횟수

count = 0
for i in range(0, 10000):
    data = train_data[i].tolist()
    predictions = optimized_predictor.predict({"instances": np.asarray(data).tolist()})
    prediction = predictions["predictions"][0]["classes"]
    label = train_labels[i]
#     print("prediction: {}, label: {}, matched: {}".format(prediction, label, prediction == label))
    if (prediction != label):
        count += 1
count = 100 - count/10000
print("Accuracy: {}".format(count))

Accuracy: 99.966
CPU times: user 22.2 s, sys: 607 ms, total: 22.8 s
Wall time: 1min 15s


### 엔드포인트 삭제

In [19]:
optimized_predictor.delete_endpoint()